In [19]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from sqlalchemy import create_engine
import os


# OAuth token request
url = "https://accounts.zoho.com/oauth/v2/token"
params = {
    "client_id": "1000.8KP5FVR2CASRD70V5D6EN3EHNZXC0W",
    "client_secret": "2e76b56ef1a4ff32d4ae87f35024be17da3e641f9e",
    "grant_type": "client_credentials",
    "scope": "ZohoAnalytics.data.read",
    "soid": "ZohoCRM.696664214"
}

response = requests.post(url, params=params)
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Token obtained successfully")
else:
    print(f"Failed to obtain token. Status code: {response.status_code}")
    exit()

# Make the API request
url = "https://analyticsapi.zoho.com/api/ashutosh@raptorsupplies.com/Zoho%20CRM%20Analytics/Logistic 2?ZOHO_ACTION=EXPORT&ZOHO_OUTPUT_FORMAT=XML&ZOHO_ERROR_FORMAT=XML&ZOHO_API_VERSION=1.0"

headers = {
    "Authorization": f"Zoho-oauthtoken {access_token}"
}

response = requests.get(url, headers=headers)

# Check the response status
if response.status_code == 200:
    # Parse the XML from response.content
    root = ET.fromstring(response.content)

    # Initialize a list to hold row data
    rows_data = []

    # Extract data from each row
    for row in root.findall('.//row'):
        # Create a dictionary for each row's data
        row_data = {}
        for column in row:
            column_name = column.attrib.get('name')  # Get the 'name' attribute as column name
            row_data[column_name] = column.text  # Get the text content of the column
        rows_data.append(row_data)

    # Create a DataFrame from the list of rows
df = pd.DataFrame(rows_data)
print(df)

Token obtained successfully
       SNo      Version Sheet.Stage  Date tracking Enter      Invoice  \
0    46794                  TRACK 2  01/03/2024 14:46:27  UK19670|OAU   
1    46993  CANCELLED RAPTOR ORDERS  02/03/2024 17:17:21  UK20004|QCD   
2    47386                  TRACK 3  07/03/2024 04:31:08  NL10750|QCD   
3    47393  CANCELLED RAPTOR ORDERS  07/03/2024 12:15:13  UK20217|QCD   
4    47394                  TRACK 1  07/03/2024 12:20:02  UK18528|QPX   
..     ...                      ...                  ...          ...   
943  60976                  TRACK 1  24/06/2024 18:26:25  NL12344|QCX   
944  60989                  TRACK 1  24/06/2024 19:25:02  UK23173|QAX   
945  60999                  TRACK 1  24/06/2024 19:57:01  UK23401|QCD   
946  61008                  TRACK 1  24/06/2024 23:56:47  UK23385|OAD   
947  61010                  TRACK 1  24/06/2024 23:58:43   AU4575|OAD   

              Supplier Name  \
0                KSC DIRECT   
1                      ebay   
2 

In [20]:
df=df[df['Version Sheet.Stage']!='ORDER CLOSED']

In [21]:
df[['Pic', 'Item Pics', 'Weight_LBS', 'Dimensions_Inches', 'Remark by Robert', 'Remark by Logistic team', 'Delivery Date']] = None
cols = list(df.columns)
tracking_index = cols.index('Tracking Number')
cols.insert(tracking_index + 1, cols.pop(cols.index('Delivery Date')))
df = df[cols]

In [22]:
df['SNo']=df.SNo.astype(int)

In [23]:
# df['SNo'] = range(1, len(df) + 1)

# # Move 'SNo' to the first position
# cols = ['SNo'] + [col for col in df.columns if col != 'SNo']
# df = df[cols]

In [24]:
df['Purchase Cost'] = df['Purchase Cost'].astype(float)
df['Pic'] = df['Purchase Cost'].apply(lambda x: 'Yes' if x > 50 else 'No')
df['Item Pics'] = df['Purchase Cost'].apply(lambda x: 'Yes' if x > 1500 else 'No')
df['Purchase Cost'] = df['Purchase Cost'].astype(str)


In [25]:
df['Date tracking Enter'] = pd.to_datetime(df['Date tracking Enter'], format='%d/%m/%Y %H:%M:%S')

# Sort the DataFrame by 'Date tracking Enter' in ascending order
df_sorted = df.sort_values(by='Date tracking Enter', ascending=True)

# Extract the earliest instance of each unique 'Invoice'
earliest_invoices = df_sorted.drop_duplicates(subset='Invoice', keep='first')

# Extract the remaining rows with 'Invoice' values
remaining_invoices = df_sorted[df_sorted.duplicated(subset='Invoice', keep='first')]

# Create an empty DataFrame to store the final sorted result
df_final = pd.DataFrame(columns=df.columns)

# Append remaining invoices just below their respective earliest invoices
for invoice in earliest_invoices['Invoice']:
    # Append the earliest invoice row
    df_final = pd.concat([df_final, earliest_invoices[earliest_invoices['Invoice'] == invoice]])
    # Append the remaining invoice rows
    df_final = pd.concat([df_final, remaining_invoices[remaining_invoices['Invoice'] == invoice]])

# Append rows without 'Invoice' values (if needed)
df_no_invoices = df_sorted[df_sorted['Invoice'].isna()]
df_final = pd.concat([df_final, df_no_invoices])

# Reset index for the final DataFrame
df_final = df_final.reset_index(drop=True)

# Display the final sorted DataFrame
print(df_final)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_9476\1178887668.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, earliest_invoices[earliest_invoices['Invoice'] == invoice]])


       SNo      Version Sheet.Stage Date tracking Enter      Invoice  \
0    46794                  TRACK 2 2024-03-01 14:46:27  UK19670|OAU   
1    46993  CANCELLED RAPTOR ORDERS 2024-03-02 17:17:21  UK20004|QCD   
2    47386                  TRACK 3 2024-03-07 04:31:08  NL10750|QCD   
3    50573                  TRACK 3 2024-04-05 09:39:51  NL10750|QCD   
4    60182                  TRACK 3 2024-06-18 19:11:34  NL10750|QCD   
..     ...                      ...                 ...          ...   
943  60969                  TRACK 1 2024-06-24 17:49:32  UK23302|OAD   
944  60989                  TRACK 1 2024-06-24 19:25:02  UK23173|QAX   
945  60999                  TRACK 1 2024-06-24 19:57:01  UK23401|QCD   
946  61008                  TRACK 1 2024-06-24 23:56:47  UK23385|OAD   
947  61010                  TRACK 1 2024-06-24 23:58:43   AU4575|OAD   

          Supplier Name Supplier (Grainger / Non-Grainger)  \
0            KSC DIRECT                         KSC DIRECT   
1          

In [8]:
# df_final.iloc[:851]

In [9]:
# df_final[df_final['SNo']>60543]

**CREATING SHEET**

In [26]:
# import pandas as pd
# import numpy as np
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# # Convert 'Timestamp' to string to avoid JSON serialization issues
# df_final['Date tracking Enter'] = df_final['Date tracking Enter'].astype(str)

# # Function to authenticate and get the Google Sheets client
# def authenticate_google_sheets(json_credentials_path):
#     scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
#     creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
#     client = gspread.authorize(creds)
#     return client

# # Function to create or update a worksheet and upload DataFrame to it in chunks
# def create_or_update_worksheet(df, sheet_name, worksheet_name, json_credentials_path):
#     try:
#         # Authenticate and get the Google Sheets client
#         client = authenticate_google_sheets(json_credentials_path)
#         print("Authenticated successfully")

#         # Open the existing Google Sheet by name
#         sheet = client.open(sheet_name)
#         print(f"Opened Google Sheet: {sheet_name}")

#         # Check if the worksheet already exists
#         try:
#             worksheet = sheet.worksheet(worksheet_name)
#             print(f"Worksheet {worksheet_name} already exists, updating it.")
#         except gspread.exceptions.WorksheetNotFound:
#             # Add a new worksheet if it does not exist
#             worksheet = sheet.add_worksheet(title=worksheet_name, rows="10000", cols="30")
#             print(f"Created new worksheet: {worksheet_name}")

#         # Convert DataFrame to list of lists
#         df_list = df.values.tolist()
#         print(f"Converted DataFrame to list of lists")

#         # Add header (column names)
#         df_list.insert(0, df.columns.tolist())
#         print(f"Inserted column headers")

#         # Clear the worksheet before updating it
#         worksheet.clear()
#         print(f"Cleared existing worksheet data")

#         # Upload data in chunks
#         chunk_size = 1000  # Adjust chunk size as needed
#         for i in range(0, len(df_list), chunk_size):
#             chunk = df_list[i:i + chunk_size]
#             cell_range = f'A{i + 1}'
#             print(f"Updating worksheet with chunk starting at {cell_range}...")
#             worksheet.update(cell_range, chunk)
#             print(f"Updated worksheet with chunk starting at {cell_range}")

#     except gspread.exceptions.APIError as api_error:
#         print(f"Google Sheets API error: {api_error}")
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")

# # Example usage
# if __name__ == "__main__":
#     # Path to the JSON credentials file
#     json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

#     # Name of the existing Google Sheet
#     sheet_name = 'logistic_CLT'

#     # Name of the worksheet
#     worksheet_name = 'Test7'

#     # Upload the DataFrame to the worksheet in the existing Google Sheet
#     create_or_update_worksheet(df_final, sheet_name, worksheet_name, json_credentials_path)
#     print("Process completed")

Authenticated successfully
Opened Google Sheet: logistic_CLT
Worksheet Test7 already exists, updating it.
Converted DataFrame to list of lists
Inserted column headers
Cleared existing worksheet data
Updating worksheet with chunk starting at A1...


C:\Users\lenovo\AppData\Local\Temp\ipykernel_9476\1562265514.py:53: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(cell_range, chunk)


Updated worksheet with chunk starting at A1
Process completed


**CHECKING ROWS TO APPENDS**

In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Function to authenticate and get the Google Sheets client
def authenticate_google_sheets(json_credentials_path):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)
    return client

# Function to get the maximum 'Sno' value from an existing worksheet
def get_max_sno(sheet_name, worksheet_name, json_credentials_path):
    try:
        # Authenticate and get the Google Sheets client
        client = authenticate_google_sheets(json_credentials_path)
        print("Authenticated successfully")

        # Open the existing Google Sheet by name
        sheet = client.open(sheet_name)
        print(f"Opened Google Sheet: {sheet_name}")

        # Check if the worksheet exists
        try:
            worksheet = sheet.worksheet(worksheet_name)
            print(f"Worksheet {worksheet_name} exists.")
        except gspread.exceptions.WorksheetNotFound:
            print(f"Worksheet {worksheet_name} does not exist.")
            return None

        # Get the maximum value in the 'Sno' column
        existing_data = worksheet.col_values(1)  # Assuming 'Sno' is in the first column
        if existing_data and existing_data[0] == 'SNo':
            existing_data = existing_data[1:]  # Remove header if present

        max_sno = max([int(sno) for sno in existing_data if sno.isdigit()], default=0)
        print(f"Maximum 'Sno' found: {max_sno}")
        return max_sno

    except gspread.exceptions.APIError as api_error:
        print(f"Google Sheets API error: {api_error}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Function to filter DataFrame based on max 'Sno' and return the new DataFrame
def get_new_rows_to_append(df, max_sno):
    new_rows = df[df['SNo'] > max_sno]
    if new_rows.empty:
        print("No new data to append")
    else:
        print(f"New rows to append: {len(new_rows)}")
    return new_rows

# Example usage
if __name__ == "__main__":
    # Path to the JSON credentials file
    json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

    # Name of the existing Google Sheet
    sheet_name = 'Tracking Sheet for Charlotte.xlsx'

    # Name of the worksheet
    worksheet_name = 'Test'

    # Create a sample DataFrame
    # data = {
    #     'SNo': [1, 2, 3, 4],  # Assuming these are the 'SNo' values
    #     'Column1': ['Value1', 'Value2', 'Value3', 'Value4'],
    #     'Column2': ['Value5', 'Value6', 'Value7', 'Value8']
    # }
    # df = pd.DataFrame(data)

    # Get the maximum 'Sno' f
    # Get the maximum 'Sno' from the worksheet
    max_sno = get_max_sno(sheet_name, worksheet_name, json_credentials_path)

    if max_sno is not None:
        # Get the new rows to append
        new_rows_df = get_new_rows_to_append(df, max_sno)
        print(new_rows_df)


Authenticated successfully
Opened Google Sheet: Tracking Sheet for Charlotte.xlsx
Worksheet Test exists.
Maximum 'Sno' found: 60969
New rows to append: 6
       SNo Version Sheet.Stage Date tracking Enter      Invoice  \
942  60975             TRACK 1 2024-06-24 18:13:00  NL12424|QAD   
943  60976             TRACK 1 2024-06-24 18:26:25  NL12344|QCX   
944  60989             TRACK 1 2024-06-24 19:25:02  UK23173|QAX   
945  60999             TRACK 1 2024-06-24 19:57:01  UK23401|QCD   
946  61008             TRACK 1 2024-06-24 23:56:47  UK23385|OAD   
947  61010             TRACK 1 2024-06-24 23:58:43   AU4575|OAD   

          Supplier Name                 Supplier (Grainger / Non-Grainger)  \
942              Amazon                                             amazon   
943    webstaurantstore  webstaurantstore\n\nPlease note @Shivansh Shek...   
944  hvacandtoolsdirect                                 hvacandtoolsdirect   
945   thedrainagesource                                  thedrai

In [11]:
import requests
import json
import pandas as pd

# FedEx Production API credentials
CLIENT_ID = 'l7da4097b7bcfe4440ba7f860d7af9bf89'
CLIENT_SECRET = '900efa8a-0fdf-420e-9aaa-27efadea3bc3'

# Function to get access token
def get_access_token(client_id, client_secret):
    url = "https://apis.fedex.com/oauth/token"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        token_info = response.json()
        access_token = token_info['access_token']
        return access_token
    else:
        print(f"Failed to get access token: {response.status_code}")
        print(response.text)
        return None

# Function to track a single shipment and fetch expected delivery date
def track_shipment(tracking_number, access_token):
    url = "https://apis.fedex.com/track/v1/trackingnumbers"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
    }
    data = {
        "trackingInfo": [
            {
                "trackingNumberInfo": {
                    "trackingNumber": tracking_number
                }
            }
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        tracking_info = response.json()
        print(f"Full response for tracking number {tracking_number}:\n", json.dumps(tracking_info, indent=2))  # Print full response for debugging
        
        try:
            complete_track_results = tracking_info['output']['completeTrackResults'][0]['trackResults'][0]
            
            # Check for estimated delivery date
            estimated_delivery = complete_track_results.get('estimatedDeliveryTimeWindow', {}).get('window', {}).get('ends')
            
            if estimated_delivery:
                return estimated_delivery
            else:
                # Fetch actual delivery date if estimated delivery date is not available
                for date_time in complete_track_results.get('dateAndTimes', []):
                    if date_time.get('type') == 'ACTUAL_DELIVERY':
                        return date_time.get('dateTime')
            return None
        except KeyError as e:
            print(f"Unexpected response structure or missing key: {e}")
            print(json.dumps(tracking_info, indent=2))  # Print full response for debugging
            return None
    else:
        print(f"Failed to track shipment: {response.status_code}")
        print(response.text)
        return None

# Example DataFrame
# data = {
#     'Tracking Number': ['730273739274', '362918040', '402458566008']
# }
# new_rows_df = pd.DataFrame(data)

# Add an empty 'Delivery Date' column
new_rows_df['Delivery Date'] = ''

# Print DataFrame columns to debug
print("DataFrame columns:", new_rows_df.columns)

# Step 1: Generate access token
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

# Step 2: Iterate over DataFrame and update 'Delivery Date' for all shipments
if access_token:
    for index, row in new_rows_df.iterrows():
        print(f"Processing row {index}: {row}")  # Debug statement to print row being processed
        tracking_numbers_str = row.get('Tracking Number')
        
        if tracking_numbers_str:
            tracking_numbers = tracking_numbers_str.split(';')
            dates = []
            
            for tracking_number in tracking_numbers:
                print(f"Tracking number: {tracking_number}")  # Debug statement to print tracking number
                delivery_date = track_shipment(tracking_number, access_token)
                if delivery_date:
                    dates.append(delivery_date)
                    print(f"Updated delivery date for tracking number {tracking_number}: {delivery_date}")  # Debug statement to print updated date
                else:
                    print(f"No delivery date found for tracking number {tracking_number}")
            
            new_rows_df.at[index, 'Delivery Date'] = ' | '.join(dates)

# Print the updated DataFrame
print(new_rows_df)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_9476\398379946.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_rows_df['Delivery Date'] = ''


DataFrame columns: Index(['SNo', 'Version Sheet.Stage', 'Date tracking Enter', 'Invoice',
       'Supplier Name', 'Supplier (Grainger / Non-Grainger)',
       'Version Sheet.Placed the Order with Supplier', 'Tracking Number',
       'Delivery Date', 'Tracking Courier Details.Courier  API List',
       'Tracking Courier Details.Tracking Destination',
       'Version Sheet.Destination Point', 'Purchase Cost',
       'Version Sheet.Order Payment Received Status',
       'Version Sheet.Date of Funds Received',
       'Version Sheet.Expected Shipment Date',
       'Version Sheet.Date of Order Received', 'Pic', 'Item Pics',
       'Weight_LBS', 'Dimensions_Inches', 'Remark by Robert',
       'Remark by Logistic team'],
      dtype='object')
Processing row 942: SNo                                                            60975
Version Sheet.Stage                                          TRACK 1
Date tracking Enter                              2024-06-24 18:13:00
Invoice                     

In [12]:
new_rows_df

,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
942,60975,TRACK 1,2024-06-24 18:13:00,NL12424|QAD,Amazon,amazon,Yes,TBA313997851557,,AMAZON,...,PAID,10/06/2024,5-7 DAYS,06/06/2024,Yes,No,None,None,None,None
943,60976,TRACK 1,2024-06-24 18:26:25,NL12344|QCX,webstaurantstore,webstaurantstore\n\nPlease note @Shivansh Shek...,Yes,777008389964,2024-06-26T14:15:00-04:00,FEDEX,...,NOT PAID,None,9-10 WEEKS,03/06/2024,Yes,Yes,None,None,None,None
944,60989,TRACK 1,2024-06-24 19:25:02,UK23173|QAX,hvacandtoolsdirect,hvacandtoolsdirect,Yes,276258131600,,FEDEX,...,PAID,17/06/2024,1-2 WEEKS,14/06/2024,Yes,No,None,None,None,None
945,60999,TRACK 1,2024-06-24 19:57:01,UK23401|QCD,thedrainagesource,thedrainagesource,Yes,1ZEW21850312049873,,UPS,...,NOT PAID,None,5-7 DAYS,21/06/2024,Yes,No,None,None,None,None
946,61008,TRACK 1,2024-06-24 23:56:47,UK23385|OAD,toolup,toolup,Yes,1Z8723500321347454,,UPS,...,PAID,21/06/2024,Express (5 - 7 days),20/06/2024,No,No,None,None,None,None
947,61010,TRACK 1,2024-06-24 23:58:43,AU4575|OAD,mscdirect,mscdirect,Yes,1Z901W400360475361,,UPS,...,PAID,21/06/2024,Flat Rate Shipping (5 - 7 days),18/06/2024,Yes,No,None,None,None,None


In [13]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Function to authenticate and get the Google Sheets client
def authenticate_google_sheets(json_credentials_path):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)
    return client

# Function to fetch data from a worksheet and convert it to a DataFrame
def fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path):
    try:
        # Authenticate and get the Google Sheets client
        client = authenticate_google_sheets(json_credentials_path)
        print("Authenticated successfully")

        # Open the existing Google Sheet by name
        sheet = client.open(sheet_name)
        print(f"Opened Google Sheet: {sheet_name}")

        # Get the worksheet by name
        worksheet = sheet.worksheet(worksheet_name)
        print(f"Accessing worksheet: {worksheet_name}")

        # Fetch all data from the worksheet
        data = worksheet.get_all_values()
        print("Fetched data from worksheet")

        # Convert the data to a DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])
        print("Converted data to DataFrame")

        return df

    except gspread.exceptions.APIError as api_error:
        print(f"Google Sheets API error: {api_error}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage
if __name__ == "__main__":
    # Path to the JSON credentials file
    json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

    # Name of the existing Google Sheet
    sheet_name = 'Tracking Sheet for Charlotte.xlsx'

    # Name of the worksheet
    worksheet_name = 'Test'

    # Fetch data from the worksheet into a DataFrame
    df_old = fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path)

    if df_old is not None:
        print("Data fetched successfully")
        print(df_old.head())
    else:
        print("Failed to fetch data")


Authenticated successfully
Opened Google Sheet: Tracking Sheet for Charlotte.xlsx
Accessing worksheet: Test
Fetched data from worksheet
Converted data to DataFrame
Data fetched successfully
     SNo      Version Sheet.Stage  Date tracking Enter      Invoice  \
0  46794                  TRACK 2  2024-03-01 14:46:27  UK19670|OAU   
1  46993  CANCELLED RAPTOR ORDERS  2024-03-02 17:17:21  UK20004|QCD   
2  47386                  TRACK 3  2024-03-07 04:31:08  NL10750|QCD   
3  50573                  TRACK 3  2024-04-05 09:39:51  NL10750|QCD   
4  60182                  TRACK 3  2024-06-18 19:11:34  NL10750|QCD   

  Supplier Name Supplier (Grainger / Non-Grainger)  \
0    KSC DIRECT                         KSC DIRECT   
1          ebay                  ( cancelled) ebay   
2     indsencon        indsencon + andersonprocess   
3      anderson        indsencon + andersonprocess   
4     indsencon        indsencon + andersonprocess   

  Version Sheet.Placed the Order with Supplier         Tra

In [14]:
df_old

,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
0,46794,TRACK 2,2024-03-01 14:46:27,UK19670|OAU,KSC DIRECT,KSC DIRECT,Yes,1ZRF58610394781994,,UPS,...,PAID,15/02/2024,Flat Shipping (5 - 7 days),15/02/2024,Yes,No,,,,
1,46993,CANCELLED RAPTOR ORDERS,2024-03-02 17:17:21,UK20004|QCD,ebay,( cancelled) ebay,Yes,9405508205499844047803,,USPS,...,NOT PAID,,2-3 WEEKS,28/02/2024,No,No,,,,
2,47386,TRACK 3,2024-03-07 04:31:08,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204970878331,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
3,50573,TRACK 3,2024-04-05 09:39:51,NL10750|QCD,anderson,indsencon + andersonprocess,Multiple Vendors,1z4e93640372781255,,UPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
4,60182,TRACK 3,2024-06-18 19:11:34,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204137800311,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,60913,TRACK 1,2024-06-24 16:16:04,AU4611|QCD,Grainger,"Grainger\n\nDDP AU, Air Freight\nGoShippo to C...",Yes,403789220777,,FEDEX,...,NOT PAID,,06-10 DAYS,20/06/2024,Yes,No,,,,
951,60938,TRACK 1,2024-06-24 16:45:59,NL12681|QAD,mscdirect,mscdirect,Yes,1Z901W400360415854,,UPS,...,PAID,19/06/2024,6-9 Days,19/06/2024,No,No,,,,
952,60940,TRACK 1,2024-06-24 16:48:24,NL12523|QAD,coasttocoastbreaker,coasttocoastbreaker,Yes,1Z4432310340263410,,UPS,...,PAID,21/06/2024,8 DAYS,12/06/2024,Yes,No,,,,
953,60947,TRACK 1,2024-06-24 16:56:43,SG4530|QCX,lowes,lowes,Yes,735011871874,,FEDEX,...,NOT PAID,,05-07 DAYS,20/06/2024,No,No,,,,


In [15]:
df_combined = pd.concat([df_old, new_rows_df], ignore_index=True)


In [16]:
df_combined

,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
0,46794,TRACK 2,2024-03-01 14:46:27,UK19670|OAU,KSC DIRECT,KSC DIRECT,Yes,1ZRF58610394781994,,UPS,...,PAID,15/02/2024,Flat Shipping (5 - 7 days),15/02/2024,Yes,No,,,,
1,46993,CANCELLED RAPTOR ORDERS,2024-03-02 17:17:21,UK20004|QCD,ebay,( cancelled) ebay,Yes,9405508205499844047803,,USPS,...,NOT PAID,,2-3 WEEKS,28/02/2024,No,No,,,,
2,47386,TRACK 3,2024-03-07 04:31:08,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204970878331,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
3,50573,TRACK 3,2024-04-05 09:39:51,NL10750|QCD,anderson,indsencon + andersonprocess,Multiple Vendors,1z4e93640372781255,,UPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
4,60182,TRACK 3,2024-06-18 19:11:34,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204137800311,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,60976,TRACK 1,2024-06-24 18:26:25,NL12344|QCX,webstaurantstore,webstaurantstore\n\nPlease note @Shivansh Shek...,Yes,777008389964,2024-06-26T14:15:00-04:00,FEDEX,...,NOT PAID,None,9-10 WEEKS,03/06/2024,Yes,Yes,None,None,None,None
957,60989,TRACK 1,2024-06-24 19:25:02,UK23173|QAX,hvacandtoolsdirect,hvacandtoolsdirect,Yes,276258131600,,FEDEX,...,PAID,17/06/2024,1-2 WEEKS,14/06/2024,Yes,No,None,None,None,None
958,60999,TRACK 1,2024-06-24 19:57:01,UK23401|QCD,thedrainagesource,thedrainagesource,Yes,1ZEW21850312049873,,UPS,...,NOT PAID,None,5-7 DAYS,21/06/2024,Yes,No,None,None,None,None
959,61008,TRACK 1,2024-06-24 23:56:47,UK23385|OAD,toolup,toolup,Yes,1Z8723500321347454,,UPS,...,PAID,21/06/2024,Express (5 - 7 days),20/06/2024,No,No,None,None,None,None


In [17]:
# Convert 'Date tracking Enter' to datetime with the correct format
df_combined['Date tracking Enter'] = pd.to_datetime(df_combined['Date tracking Enter'], format='%Y-%m-%d %H:%M:%S')

# Sort the DataFrame by 'Date tracking Enter' in ascending order
df_sorted = df_combined.sort_values(by='Date tracking Enter', ascending=True)

# Extract the earliest instance of each unique 'Invoice'
earliest_invoices = df_sorted.drop_duplicates(subset='Invoice', keep='first')

# Extract the remaining rows with 'Invoice' values
remaining_invoices = df_sorted[df_sorted.duplicated(subset='Invoice', keep='first')]

# Create an empty DataFrame to store the final sorted result
df_final = pd.DataFrame(columns=df.columns)

# Append remaining invoices just below their respective earliest invoices
for invoice in earliest_invoices['Invoice']:
    # Append the earliest invoice row
    df_final = pd.concat([df_final, earliest_invoices[earliest_invoices['Invoice'] == invoice]])
    # Append the remaining invoice rows
    df_final = pd.concat([df_final, remaining_invoices[remaining_invoices['Invoice'] == invoice]])

# Append rows without 'Invoice' values (if needed)
df_no_invoices = df_sorted[df_sorted['Invoice'].isna()]
df_final = pd.concat([df_final, df_no_invoices])

# Reset index for the final DataFrame
df_final = df_final.reset_index(drop=True)

# Display the final sorted DataFrame
print(df_final)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_9476\4180061375.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, earliest_invoices[earliest_invoices['Invoice'] == invoice]])


       SNo      Version Sheet.Stage Date tracking Enter      Invoice  \
0    46794                  TRACK 2 2024-03-01 14:46:27  UK19670|OAU   
1    46993  CANCELLED RAPTOR ORDERS 2024-03-02 17:17:21  UK20004|QCD   
2    47386                  TRACK 3 2024-03-07 04:31:08  NL10750|QCD   
3    50573                  TRACK 3 2024-04-05 09:39:51  NL10750|QCD   
4    60182                  TRACK 3 2024-06-18 19:11:34  NL10750|QCD   
..     ...                      ...                 ...          ...   
956  60969                  TRACK 1 2024-06-24 17:49:32  UK23302|OAD   
957  60989                  TRACK 1 2024-06-24 19:25:02  UK23173|QAX   
958  60999                  TRACK 1 2024-06-24 19:57:01  UK23401|QCD   
959  61008                  TRACK 1 2024-06-24 23:56:47  UK23385|OAD   
960  61010                  TRACK 1 2024-06-24 23:58:43   AU4575|OAD   

          Supplier Name Supplier (Grainger / Non-Grainger)  \
0            KSC DIRECT                         KSC DIRECT   
1          

In [18]:
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# Convert 'Timestamp' to string to avoid JSON serialization issues
df_final['Date tracking Enter'] = df_final['Date tracking Enter'].astype(str)

# Function to authenticate and get the Google Sheets client
def authenticate_google_sheets(json_credentials_path):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)
    return client

# Function to create or update a worksheet and upload DataFrame to it in chunks
def create_or_update_worksheet(df, sheet_name, worksheet_name, json_credentials_path):
    try:
        # Authenticate and get the Google Sheets client
        client = authenticate_google_sheets(json_credentials_path)
        print("Authenticated successfully")

        # Open the existing Google Sheet by name
        sheet = client.open(sheet_name)
        print(f"Opened Google Sheet: {sheet_name}")

        # Check if the worksheet already exists
        try:
            worksheet = sheet.worksheet(worksheet_name)
            print(f"Worksheet {worksheet_name} already exists, updating it.")
        except gspread.exceptions.WorksheetNotFound:
            # Add a new worksheet if it does not exist
            worksheet = sheet.add_worksheet(title=worksheet_name, rows="10000", cols="30")
            print(f"Created new worksheet: {worksheet_name}")

        # Convert DataFrame to list of lists
        df_list = df.values.tolist()
        print(f"Converted DataFrame to list of lists")

        # Add header (column names)
        df_list.insert(0, df.columns.tolist())
        print(f"Inserted column headers")

        # Clear the worksheet before updating it
        worksheet.clear()
        print(f"Cleared existing worksheet data")

        # Upload data in chunks
        chunk_size = 1000  # Adjust chunk size as needed
        for i in range(0, len(df_list), chunk_size):
            chunk = df_list[i:i + chunk_size]
            cell_range = f'A{i + 1}'
            print(f"Updating worksheet with chunk starting at {cell_range}...")
            worksheet.update(cell_range, chunk)
            print(f"Updated worksheet with chunk starting at {cell_range}")

    except gspread.exceptions.APIError as api_error:
        print(f"Google Sheets API error: {api_error}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage
if __name__ == "__main__":
    # Path to the JSON credentials file
    json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

    # Name of the existing Google Sheet
    sheet_name = 'Tracking Sheet for Charlotte.xlsx'

    # Name of the worksheet
    worksheet_name = 'Test'

    # Upload the DataFrame to the worksheet in the existing Google Sheet
    create_or_update_worksheet(df_final, sheet_name, worksheet_name, json_credentials_path)
    print("Process completed")

Authenticated successfully
Opened Google Sheet: Tracking Sheet for Charlotte.xlsx
Worksheet Test already exists, updating it.
Converted DataFrame to list of lists
Inserted column headers
Cleared existing worksheet data
Updating worksheet with chunk starting at A1...


C:\Users\lenovo\AppData\Local\Temp\ipykernel_9476\1552257675.py:53: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(cell_range, chunk)


Updated worksheet with chunk starting at A1
Process completed


In [44]:
df_final[df_final.duplicated()]

,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
